In [1]:
!pip install -q -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers unsloth accelerate streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 49.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 57.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.6 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.1 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 10.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 24.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 13.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 8.2 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.6 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.9 MB/s eta 0:00:00


In [2]:
# import torch
# import re
# import gc
# from unsloth import FastLanguageModel
# from transformers import AutoTokenizer, pipeline

# def load_model():
#     base_model_path = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
#     peft_model_id = "/kaggle/input/automated-essay-scoring-unsloth-finetune/trained_weights"

#     max_seq_length = 2048  # Match your training seq length
#     dtype = None  # Auto-detect
#     load_in_4bit = True  # Use 4-bit quantization

#     # 🔹 Load base model using Unsloth
#     model, tokenizer = FastLanguageModel.from_pretrained(
#         model_name=base_model_path,
#         max_seq_length=max_seq_length,
#         dtype=dtype,
#         load_in_4bit=load_in_4bit,
#     )

#     # 🔹 Load LoRA adapters (Do NOT reapply `get_peft_model`)
#     model.load_adapter(peft_model_id)

#     # 🔹 Optimize for inference (IMPORTANT)
#     FastLanguageModel.for_inference(model)

#     # 🔹 Ensure model is in evaluation mode
#     model.eval()

#     # 🔹 Set padding token
#     tokenizer.pad_token_id = tokenizer.eos_token_id

#     # 🔹 Add back `pipeline()`
#     generator = pipeline(
#         task="text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
#     )

#     return model, tokenizer, generator

# # Load model & tokenizer
# model, tokenizer, generator = load_model()




In [3]:

# def generate_prompt(essay_text):
#     """Forces structured output with Score and Reasoning clearly separated."""
#     return f"""
#     You are an expert essay evaluator. Read the essay below and evaluate it using the Scoring Criteria.

#     ### **Scoring Criteria:**
#     - **6 (Excellent):** Insightful, well-structured, strong evidence, minimal errors.
#     - **5 (Good):** Clear, logical, solid evidence, mostly error-free.
#     - **4 (Satisfactory):** Adequate reasoning, relevant examples, minor issues.
#     - **3 (Developing):** Some reasoning, weak structure, noticeable errors.
#     - **2 (Poor):** Vague argument, poor organization, frequent errors.
#     - **1 (Very Weak):** No clear argument, incoherent, major issues.

#     **Essay:**
#     "{essay_text}"

#     **Assign a final score (1 to 6) based on the criteria above.**
#     - **Return the number first then the explanation.**

#     **Final Score:** 
#     """.strip()

        
# def predict_score(essay_text):
#     """Generates a predicted score and reasoning for an essay using `pipeline()` for better memory handling."""
#     prompt = generate_prompt(essay_text)

#     # 🔹 Dynamically adjust max_new_tokens based on input essay length
#     max_tokens = max(100, min(150, int(len(essay_text.split()) * 0.5)))

#     # ✅ Run inference with `pipeline()`
#     with torch.no_grad():  # Prevents VRAM spike
#         result = generator(
#             prompt,
#             max_new_tokens=max_tokens,  # 🔹 Dynamically adjusted
#             temperature=0.1,
#             do_sample=False
#         )

#     generated_text = result[0]['generated_text'].strip()

#     print("\n==== GENERATED TEXT ====\n")
#     print(generated_text)
#     print("\n========================\n")

#     # ✅ Extract score (1-6) from "**Assigned Score:** X"
#     score_match = re.search(r"\*\*Final Score:\*\*\s*(\d)\b", generated_text)
#     score = int(score_match.group(1)) if score_match else "Undetermined"

#     # ✅ Extract reasoning from "**Explanation:** ..."
#     reasoning_match = re.search(r"\*\*Explanation:\*\*\s*(.*)", generated_text, re.DOTALL)
#     reasoning = " ".join(re.split(r'(?<=[.!?])\s+', reasoning_match.group(1).strip())[:3]) if reasoning_match else "No reasoning provided."

#     # ✅ Free up memory
#     torch.cuda.empty_cache()
#     gc.collect()

#     return score, reasoning


In [4]:
# # 1
# essay_text = "There is an advantage of limiting car usage, it prevents pollution and smog. It makes your life better in Vauban, Germany a residents of this upscale community aew suburban pioneers, where people have given up their cars. Automobiles are the linchpin of suburbs, where middle-class families from chicago to Shanghai tend to make their homes.The residents are trying to make cities denser, and better for walking for the past two decades, so Vauban, home to 4,400 residents within a rectangularsquare mile, may be the most advanced experiment in low-car suburban life. In United States, the Environmental protection Agency is promoting ""car reduced"" communities, and legislators are starting to act, if cautiously.Paris enforced a partial driving ban to clear the air of the global city. It is either leave your cars at home or suffer with a fine of 22-euro on mondays with even-numbered license plates and the same with the odd-numbered plates the following day. Because Paris typically has more smog than other European, delivery companies complained of lost revenue, while exceptions were made for plug-in cars, hybrids, and cars carrying 3 or more passengers.A lot of Americans are buying less cars, a study last year found out that driving by young people decresed 23 percent betweeen 2001-2009. Executive chairman of the Ford Motor Company, laid out a business plan for a world in which personal vehicle ownership is impractical."

In [5]:
# # 2
# essay_text = "cars, they wernt invented in the old days. who needs them, not me. screw cars. Ill tell u what u need to know about life and transportation. We can limit car use if u just use this information and use it in your life.paris has so much smog. Ew!!! No one likes that so lets limit car use. paris has 147 micrograms of matter in cubic meter measurement. Dang taxes! I hate soccer moms, my mom is one so i cant hate them too much. All they do is drive everywhere and use up all the resources so there kid can strike out in little leauge! Now i will tell you how to fix this problem.if some of the stupid soccer moms driving around there rolling egg (mini van) the car use would be cut in half. Then you might say then what is my little kid going to do. You can read a biycle places just like it said in line 24 of ""car-free day spinning into a big hit bogota. Now ill tell you what kind of bike and what your gonna do with it and your gonna love it!A bmx bike is what im talking about. you can transportate it to the skatepark to ride the ramps. Its really fun and i love it. Those gas using Mini vans would be off the streets and bmx bikes can take over. Think about our earth for once please.well to conclude things up car use is oversused highly. It is mainly the soccer moms fault. To fix all the problems in the world everyone needs to get a bmx bike and go ride some dirt jumps and this world would be a happier place!!!"


In [6]:
# # 3
# essay_text = "a time in the futur where no one needs to buy cars because there will be a fleet of driverless cars is not a safe futur. In paragraph nine the article says ""most driving laws focus on keeping drivers, passengers, and pedestrians safe, and lawmakers know that safety is best achieved with alert drivers."" Making a driverless car removes the alert driver wright out of the eqation. And a car without an alert driver isnt a safe car, at least not in law makers eyes.In states like California, Nevada, Florida, and the district of columbia it is illegal to test computer driven cars. The rest of america should follow suit though the article says, "" manufactures believe that more states will follow as soon as the cars are proved more reliably safe."" For the time being there not safe enough. though there are constant development being made to the driverless car. There are still to many hole in the ideal to make a safe apperance on the road.one of the advancements is that the cars have ""Traffic Jam Asstancet."" the article say they can steer, accelerate, and brake themselves, but all are designed to notify the driver when the road ahead requires human skills, such as navigating through work zones and around accidents."" If the driverless car cant handle a workzone or accident. Then it diffinitly can't handle roads with wondering children and dogs.drivers get distracted while driving there self driven cars. they couldn't handle a driverless one.    "


In [7]:
# # 4
# essay_text = "Space is an interesting and confusing large mass of black with so much in it to explore. Who wouldn't want the opportunity to explore it? The author of""The Challenge of Exploring Venus"" is very for the exploration of Venus. Venus is a planet in our solar system. It is sometimes called the evening star because it is one of the brightest points in the night sky or it is called Earth's twin due to similarity in density and size. We also dont know much about Venus because it is very dangerous due to very hot temperatures and many other things. But exploring it would be very worth it. Studying Venus is a worthy pursuit despite the dangers it presents beacause human curiosity will lead us into many equally intimidating endeavors, we could get more insight about Venus, and our travels on Earth, and beyond should not be limited by dangers and doubts.One reason that studying Venus is a worthy pursuit despite the dangers it presents is beacuse human curiousity will lead us into many equally intimidating endeavors. Who knows, human curiousity and advancing in technology could lead us to any other planet in our solar system. The more we know about life the more we can explore. In the story it says, "" These differences in speed mean that sometimes we are closer to Mars and other times to Venus."" If we explore Venus we can most definitely explore Mars as well.Another reason that studying Venus is worthy is beacuse we can get more insight about Venus. We need more information about Venus because it is Earth's sister planet. The more we understand Venus the easier it will be to understand space. In the text it says, "" If our sister planet is so inhospititable, why are scientists even discussing further visits to its surface"" This could possibly mean we could live on Venus someday. That would be a major benefit if anything ever happened to Earth.The third reson that studying Venus is worthy despite the dangers is because our travels on earth and beyond should not be limited by dangers and doubt. If we don't explore there is no explanations to peoples questions. Everyday we risk our lives because we never know what can happen. We shouldn't limit our decisions and our exploration based on fear. In the text it says "" just as ariplanes travel at a high altitude"" which means that it wouldnt be anything super unsafe to hover above Venus so why not try and do it.In conclusion, studying Venus is a worthy pursuit despite the dangers it presents.If we explored it human curiousity will lead us into many equally intimidating endeavors, we could learn more about venus, and our travels on Earth and beyond should not be limited by our dangers and doubts. Venus is a planet that is very close and similar to Earth why should we not explore it?So much good would come out of the exploration of Venus. The danger of anything shouldn't stop people from exploring.                    "


In [8]:
# # 5
# essay_text = "Since World War II we have been heavily dependent on the modern day transpotation: the car. Some say the car is an innovative and convienent method of transportation, while others say that it will be the end of the environment. As the evidence indicates, there are advantages to limiting car usage.Dissenters of the idea claim to say that limiting car usage will negatively impact the car industry, but it doesn't. Limiting car usage doesn't mean that people will have to stop buying cars; it simply means that they will have to find a different method of transporation. Source 2 says that in Paris on certain days motorist, who had an odd or even license plate number, had to leave their cars at home and find an alternative method of transportation. Does the evidence demonstrate an impact to car companies? Did they have a financial deficit due to their lack of buyers? The answer is simple: no, because they didn't ban the use of cars. Also, car companies can well benefit from places that limit car usage but allow hybrid cars; this allows for an increase in revenue. Most car companies in today's society model hybrid cars that run on primarily electricity and gas. In Paris, they made exceptions ""for plug-in cars, hybrids, and cars carrying three or more passengers (Source 2)"". These exceptions actually promote the buying of hybrid cars! This will boost the economic profit of the car companies, not damage it. Limiting car usage does not affect car companies, but instead supports it.The limitation of car usage betters the environment. Cars emit greenhouse gases in the atmosphere faster than they can be removed. This causes health problems in living organisms and pollution of ecosystems. Cars are liable for 12 percent of greenhouse gas emissions in Europe and 50 percent in areas in the U.S. as Source 1 states. People are playing with the balance of nature and if its not fixed then humans are as good as dead; this is why the implementation of limiting car usage benefits people. The effects of this can be seen in Bogota, Colombia, whose goal is to reduce smog and promote alternative transporation (Source 3), and Paris, where intense smog covered the enitre city: ""The smog rivaled Beijing, China, which is known as one of the most polluted cities in the world.(Source 2)"" The source explains that cold nights and warm days trapped in car emissions and after a few days of reducing car usage the air cleared up. This shows the effectiveness of the limiations of car usage.The limiting of cars also boosts community wellness. The people who experienced this regulation had positive views on the matter and even on rainy days they participated! Bogota, Colombia has gone into its third year of banning cars and the turnouts of the pariticipation were large, with two other Colombian cities joining and municipal authorities from other countries to view the event (Source 3). The evidence show that limiting cars wasn't opposed by the people, in fact, other countries were wanting to get in on the action. This limiting of car usage really cuts down on traffic congestion and new community projects have been established to ease the transition to alternative transportation. Heidrun Walter, mentioned in Source 1, syas that when she had her car she always tense and stressed and is happy with the change. The source also mentions that the ""swish of bicycles and the chatter of wandering children drown out the occasional distant motor (Source 1)."" This shows that the limitation of car usage greatly enchances the wellness of the community and promotes a safe and stress-free environment for the people.In summation, the limitation of car usage doesn't negatively impact car companies, but instead increases hybrid car sale revenue, echances community wellness, betters the environment. The things that can be accomplished with this regulation only has benefits for everyone. With this in mind, why not implement this ideal in order to better the lives of not only this generation but the future leaders of the world?"


In [9]:
# # 6
# essay_text = "In ""The Challenge of Exploring Venus,"" the author gives readers details on how Venus is Earth's ""twin."" Venus is the closest planet to Earth in both size and distance. The author discusses with readers why scientist want to explore Venus and how they are taking the steps to do it. The author of ""The Challenge of Exploring Venus"" supports their idea that studying Venus is a worthy pursuit despite the dangers by explaining how humans have the technology, why exploring Venus is important, and by informing readers on the dangers of exploration.First, the author supports their idea that the danger of exploring Venus well by explaining how humans do have the technology to explore Venus. The text gives examples of many advancements that have been made in order to explore Earth's sister planet. The author writes, ""For example, some simplified electronics made of silicon carbide have been tested in a chamber simulating the chaos of Venus's surface and have lasted for three weeks."" This quote shows that the author's idea is well supported because it gives an actual example of the technology that has been created in order to explore Venus. Space exploration has been advancing and improving for years, so eventually humans should be able to explore planets with the worst conditions easily. Another way that the author's claim is supported is when they discuss that the technology needed has been around since before humans even traveled to space. The text discuss a device that has been around since the 1800s and played a significant role in World War II. This goes to show that humans have more than enough resources to travel to Venus no matter how dangerous it may be. In all, the author supports the idea that exploring to Venus is worth any danger that humans could face well by giving information on the technology that humans already have to do it.Secondly, the author supports their idea that going to Venus is worth it because they discuss how important it is to explore Venus. The text discusses that out of all the planets in the solar system, Venus is most like Earth. The text reads, ""Astronomers are fascinated by Venus because it may well have once been the most Earth-like planet in our solar system. Long ago, Venus was probably covered largely with oceans and could have supported various froms of life, just like Earth."" This quote displays that the author's idea is well supported because it tells the readers how important exploring Venus is. By studying Venus, scientist could learn more about different forms of life. There could have been different plants and animals on Venus then there is now on Earth. If scientist were able to figure that out, it would be an amazing discovery and could improve the world's knowledge on space overall. Furthermore, the author talks about how much more convinient it is for humans to travel to Venus. Since space travel can take years, Venus is one of Earth's only options for exploration. So, the author's claim is well supported because they talk about how significant the exploration of Venus is for science.Finally, the author's idea is well supported because even though they support the exploration of Venus, they still discuss the possible dangers of Venus. It states repeatedly in the text that Earth's conditions greatly differ to Venus's. The text says that ""Each previous mission was unmanned, and for a good reason, since no space craft survived the landing for more than a few hours."" This shows that the author's claim is well supported because they are acknowledging the fact that the exploration of Venus is not an easy task. Sending humans to Venus can be a fatal misson, so it is a good thing that the author can explain that, while still supporting their claim. Another way the author shows the danger of exploring Venus is in parapgraph three when it reads,"" These conditons are far more extreme than anything the humans encounter on Earth; such an enviroment would crush even a submarine accustomed to diving to the deepest parts of our oceans and would liquefy many metals."" This shows that the author's idea is well supported because they still acknowledge the fact that going to Venus is extremely danger for both humans and any sort of machine. In summation, the author's idea is well supported because despite the fact that they are for the exploration of Venus, they still informed the readers on the dangers.Consequently, the author of ""The Challenge of Exploring Venus"" supports their idea that exploring Venus no matter the dangers well by explaining how humans have the technology needed, why exploring Venus is important for science, and by informing readers on the dangers of exploring Venus despite the fact that they believe that it is worth the risk. The author of this text informs readers well and has an open mind while writing since they did explain the dangers. The readers can form an opinion on this subject since the author both dicussed how exploring Venus is important and how it may be extremely dangerous to explore. "


In [10]:
# score, reasoning = predict_score(essay_text)

In [11]:
# score

In [12]:
# reasoning

In [21]:
%%writefile app.py

import streamlit as st
import torch
import re
import gc
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, pipeline

@st.cache_resource
def load_model():
    base_model_path = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    peft_model_id = "/kaggle/input/automated-essay-scoring-unsloth-finetune/trained_weights"

    max_seq_length = 2048  # Match your training seq length
    dtype = None  # Auto-detect
    load_in_4bit = True  # Use 4-bit quantization

    # 🔹 Load base model using Unsloth
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model_path,
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )

    # 🔹 Load LoRA adapters (Do NOT reapply `get_peft_model`)
    model.load_adapter(peft_model_id)

    # 🔹 Optimize for inference (IMPORTANT)
    FastLanguageModel.for_inference(model)

    # 🔹 Ensure model is in evaluation mode
    model.eval()

    # 🔹 Set padding token
    tokenizer.pad_token_id = tokenizer.eos_token_id

    # 🔹 Add back `pipeline()`
    generator = pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
    )

    return model, tokenizer, generator

# Load model & tokenizer
model, tokenizer, generator = load_model()



def generate_prompt(essay_text):
    """Forces structured output with Score and Reasoning clearly separated."""
    return f"""
    You are an expert essay evaluator. Read the essay below and evaluate it using the Scoring Criteria.

    ### **Scoring Criteria:**
    - **6 (Excellent):** Insightful, well-structured, strong evidence, minimal errors.
    - **5 (Good):** Clear, logical, solid evidence, mostly error-free.
    - **4 (Satisfactory):** Adequate reasoning, relevant examples, minor issues.
    - **3 (Developing):** Some reasoning, weak structure, noticeable errors.
    - **2 (Poor):** Vague argument, poor organization, frequent errors.
    - **1 (Very Weak):** No clear argument, incoherent, major issues.

    **Essay:**
    "{essay_text}"

    **Assign a final score (1 to 6) based on the criteria above.**
    - **Return the number first then the explanation.**

    **Final Score:** 
    """.strip()

        
def predict_score(essay_text):
    """Generates a predicted score and reasoning for an essay using `pipeline()` for better memory handling."""
    prompt = generate_prompt(essay_text)

    # 🔹 Dynamically adjust max_new_tokens based on input essay length
    max_tokens = max(100, min(150, int(len(essay_text.split()) * 0.5)))

    # ✅ Run inference with `pipeline()`
    with torch.no_grad():  # Prevents VRAM spike
        result = generator(
            prompt,
            max_new_tokens=max_tokens,  # 🔹 Dynamically adjusted
            temperature=0.1,
            do_sample=False
        )

    generated_text = result[0]['generated_text'].strip()

    print("\n==== GENERATED TEXT ====\n")
    print(generated_text)
    print("\n========================\n")

    # ✅ Extract score (1-6) from "**Final Score:** X"
    score_match = re.search(r"\*\*Final Score:\*\*\s*(\d)\b", generated_text)
    score = int(score_match.group(1)) if score_match else "Undetermined"

    # ✅ Extract reasoning from "**Explanation:** ..."
    reasoning_match = re.search(r"\*\*Explanation:\*\*\s*(.*)", generated_text, re.DOTALL)
    reasoning = " ".join(re.split(r'(?<=[.!?])\s+', reasoning_match.group(1).strip())[:3]) if reasoning_match else "No reasoning provided."

    # ✅ Free up memory
    torch.cuda.empty_cache()
    gc.collect()

    return score, reasoning


# Streamlit UI
st.markdown(
    """
    <h1 style='text-align: center; font-size: 2.5rem;'>📝 AI Essay Scorer</h1>
    <h3 style='text-align: center; color: #4CAF50;'>Powered by Llama-3.1 8B Instruct Fine-Tuned</h3>
    <hr style='border: 1px solid #ddd;'>
    """,
    unsafe_allow_html=True
)

# Essay Input Section
st.markdown("**Enter an essay below to receive an AI-generated score (1 to 6) with reasoning:**")
essay_text = st.text_area("Paste your essay here:", height=300, placeholder="Start typing your essay...")

# Scoring Button
col1, col2, col3 = st.columns([1, 2, 1])
with col2:
    evaluate_btn = st.button("📊 Evaluate Essay", use_container_width=True)

if evaluate_btn:
    if essay_text.strip():
        with st.spinner("Evaluating... Please wait."):
               score, reasoning = predict_score(essay_text)  # Your function to evaluate essay
       
        # Display results
        st.success(f"🏆 **Essay Score:** {score}/6")
        st.markdown("### 📌 AI's Justification:")
    
        st.markdown(
            f"""
            <div style="
                padding: 10px;
                background-color: #f9f9f9;
                border-left: 5px solid #4CAF50;
                border-radius: 5px;
                font-size: 1rem;
                line-height: 1.6;
                white-space: pre-wrap;
                word-wrap: break-word;
                text-align: justify;
            ">
                {reasoning}
            </div>
            """,
            unsafe_allow_html=True
        )

    else:
        st.warning("⚠️ Please enter an essay before evaluating.")

# Sidebar - GPU Monitoring
st.sidebar.header("System Status")
if torch.cuda.is_available():
    gpu_mem = torch.cuda.memory_allocated() / 1e9
    st.sidebar.metric(label="🖥️ GPU Memory Used", value=f"{gpu_mem:.2f} GB")
else:
    st.sidebar.info("⚠️ GPU not detected. Running on CPU.")

# Footer with logo
st.markdown(
    """
    <hr style='border: 1px solid #ddd;'>
    <div style='text-align: center;'>
        <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR6mrEiWWHVIDYf_NgaG29rI3zugoN9cFCDYQ&s" width="200">
        <p style='color: gray;'>Developed with Streamlit | AI-powered automated essay evaluation</p>
    </div>
    """,
    unsafe_allow_html=True
)


Overwriting app.py


# Running Streamlit

In [22]:
!npm install localtunnel

⠙⠹⠸⠼⠴
up to date, audited 23 packages in 715ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠴

In [23]:
# Your public ip is the password to the localtunnel
!curl ipv4.icanhazip.com

35.227.174.128


In [24]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501

⠙your url is: https://true-masks-fetch.loca.lt
^C
